In [4]:
import keras
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, concatenate
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import numpy as np
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import tensorflow as tf
#keras: v2.1.6 --- tf: v1.8.0
print('Keras version: ', keras.__version__)
print('Tensorflow version: ', tf.__version__)
%matplotlib inline

Keras version:  2.1.6
Tensorflow version:  1.8.0


In [14]:
face_input_shape = (227, 227, 3)
facial_image_input = Input(face_input_shape)


#11x11 conv, 96 filters, stride=4
facial_recognition = Conv2D(filters=96, kernel_size=(11,11), strides=(4,4),name = 'face_recognition_conv1')(facial_image_input)
facial_recognition = BatchNormalization(axis=3, name = 'face_recognition_bn1')(facial_recognition)
facial_recognition = Activation(PReLU())(facial_recognition)
facial_recognition = MaxPooling2D(pool_size=(3,3), strides=(2,2), name = 'face_recognition_mp1')(facial_recognition)

mediator = Conv2D(filters=96, kernel_size = (6,6), strides = (3,3), name = 'mediator_conv0')(facial_recognition)
branch_one = Conv2D(filters = 256, kernel_size = (4,4), strides = (4,4), name = 'branch_one_conv1')(mediator)    

#5x5 conv, 256 filters, stride=2
facial_recognition = Conv2D(filters=256, kernel_size=(5,5), strides=(2,2), name = 'face_recognition_conv2')(facial_recognition)
facial_recognition = BatchNormalization(axis=3, name = 'face_recognition_bn2')(facial_recognition)
facial_recognition = Activation(PReLU())(facial_recognition)
facial_recognition = MaxPooling2D(pool_size=(3,3), strides=(2,2), name = 'face_recognition_mp2')(facial_recognition)
    
#3x3 conv, 384 filters
facial_recognition = Conv2D(filters=384, kernel_size=(3,3), padding='same',name = 'face_recognition_conv3')(facial_recognition)
facial_recognition = Activation(PReLU())(facial_recognition)

branch_two = Conv2D(filters=256, kernel_size = (2,2), strides = (2,2), name = 'branch_two_conv1')(facial_recognition)    

##3x3 conv, 384 filters
facial_recognition = Conv2D(filters=384, kernel_size=(3,3), padding='same',name = 'face_recognition_conv4')(facial_recognition)
facial_recognition = Activation(PReLU())(facial_recognition)
 

#3x3 conv, 512 filters
facial_recognition = Conv2D(filters=512, kernel_size=(3,3), padding='same',name = 'face_recognition_conv5')(facial_recognition)
facial_recognition = Activation(PReLU())(facial_recognition)

branch_three = MaxPooling2D(pool_size = (3,3), strides=(2,2), name = 'branch_three_mp1')(facial_recognition)    

#3x3 conv, 512 filters
facial_recognition = Conv2D(filters=512, kernel_size=(3,3), padding='same', name = 'face_recognition_conv6')(facial_recognition)
facial_recognition = Activation(PReLU())(facial_recognition)

age_gender = MaxPooling2D(pool_size = (2,2), name = 'age_mp1')(facial_recognition)    

#3x3 conv, 512 filters
facial_recognition = Conv2D(filters=512, kernel_size=(3,3), padding='same',name = 'face_recognition_conv7')(facial_recognition)
facial_recognition = MaxPooling2D(pool_size=(3,3), strides=(2,2), name = 'face_recognition_mp7')(facial_recognition)
facial_recognition = Activation(PReLU())(facial_recognition)
    
#fully-connected layer - 1024 units
facial_recognition = Flatten()(facial_recognition)
facial_recognition = Dense(units=1024, activation=PReLU(), name = 'face_recognition_fc1')(facial_recognition)

#fully-connected layer - 512 units 
facial_recognition = Dense(units=512, activation=PReLU(), name = 'face_recognition_fc2')(facial_recognition)

#fully-connected layer - 10548 units
facial_recognition = Dense(units=10548, activation='softmax', name = 'face_recognition_fc3')(facial_recognition)

merge = concatenate([branch_one, branch_two, branch_three])
merge = Flatten()(merge)
merge = Dropout(0.5)(merge)
merge = Dense(2048, activation = PReLU(), name='branch_merge_fc1')(merge)

face_detection = Dense(512, activation=PReLU(), name='face_detection_fc1')(merge)
face_detection = Dense(2, activation='softmax', name='face_detection_fc2')(face_detection)

visibility = Dense(512, activation=PReLU(), name='visibility_fc1')(merge)
visibility = Dense(21, activation='softmax', name='visibility_fc2')(visibility)

landmarks = Dense(512, activation=PReLU(), name='landmarks_fc1')(merge)
landmarks = Dense(42, activation='softmax', name='landmarks_fc2')(landmarks)

pose_estimation = Dense(512, activation = PReLU(), name = 'pose_estimation_fc1')(merge)
pose_estimation = Dense(3, activation = 'softmax', name = 'pose_estimation_fc2')(pose_estimation)

smile = Dense(512, activation=PReLU(), name = 'smile_fc1')(merge)
smile = Dense(2, activation='softmax', name = 'smile_fc2')(smile)

age = Dense(1024, activation=PReLU(), name = 'age_fc1')(age_gender)
age = Dense(128, activation=PReLU(), name = 'age_fc2')(age)
age = Dense(1, activation='softmax', name = 'age_fc3')(age)

gender = Dense(1024, activation=PReLU(), name = 'gender_fc1')(age_gender)
gender = Dense(128, activation=PReLU(), name = 'gender_fc2')(gender)
gender = Dense(2, activation='softmax', name = 'gender_fc3')(gender)

output = [facial_recognition, face_detection, visibility, landmarks, pose_estimation, smile, age, gender]

all_in_one = Model(inputs= facial_image_input, outputs = output, name = 'All_in_one_model')

/home/nardos/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/keras/activations.py:115: UserWarning: Do not pass a layer instance (such as PReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [15]:
all_in_one.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
face_recognition_conv1 (Conv2D) (None, 55, 55, 96)   34944       input_11[0][0]                   
__________________________________________________________________________________________________
face_recognition_bn1 (BatchNorm (None, 55, 55, 96)   384         face_recognition_conv1[0][0]     
__________________________________________________________________________________________________
activation_60 (Activation)      (None, 55, 55, 96)   0           face_recognition_bn1[0][0]       
__________________________________________________________________________________________________
face_recog

In [18]:
all_in_one.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])